In [1]:
import numpy as np
import scipy as sp
import mne
from sklearn.ensemble import RandomForestClassifier
from nodestimation.learning.selection import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.project import  find_subject_dir
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from nodestimation.project.actions import read
import nodestimation as nd

In [30]:
titanic = pd.read_csv('/home/user/Downloads/titanic.csv')

titanic['Survived'] = pd.Series(surv == 1 for surv in titanic['Survived'])
titanic['Sex'] = pd.Series([int(sex == 'female') for sex in titanic["Sex"]])
for row in titanic.index:
    if not np.isfinite(titanic.loc[row]['Age']):
        titanic.loc[row, 'Age'] = titanic['Age'].mean()
titanic['Embarked'] = pd.Series({'S': 0, 'C': 1, 'Q': 2}[emb] for emb in titanic['Embarked'])
dataset = titanic.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1)
data = dataset.drop(['Survived'], axis=1)
print(data)
target = dataset['Survived']
survived = pd.DataFrame([data.iloc[i] for i in range(dataset.shape[0]) if dataset.iloc[i]['Survived']])
died = pd.DataFrame([data.iloc[i] for i in range(dataset.shape[0]) if not dataset.iloc[i]['Survived']])

statistic = collect_statistic(data)

print(compute_importance(survived, statistic))
print(compute_importance(died, statistic))

     Pclass  Sex        Age  SibSp  Parch     Fare  Embarked
0         3    0  22.000000      1      0   7.2500         0
1         1    1  38.000000      1      0  71.2833         1
2         3    1  26.000000      0      0   7.9250         0
3         1    1  35.000000      1      0  53.1000         0
4         3    0  35.000000      0      0   8.0500         0
..      ...  ...        ...    ...    ...      ...       ...
886       2    0  27.000000      0      0  13.0000         0
887       1    1  19.000000      0      0  30.0000         0
888       3    1  29.699118      1      2  23.4500         0
889       1    0  26.000000      0      0  30.0000         1
890       3    0  32.000000      0      0   7.7500         2

[891 rows x 7 columns]
Pclass      1.004065
Sex         1.157997
Age         0.768822
SibSp       0.537720
Parch       0.748418
Fare        0.730084
Embarked    0.863355
dtype: float64
Pclass      0.853621
Sex         0.828392
Age         0.653657
SibSp       0.71552

In [40]:
rforest = RandomForestClassifier()
rforest.fit(data, target)
importances_df = pd.DataFrame(rforest.feature_importances_, columns=['Feature_Importance'],
                              index=data.columns)
print(importances_df)

          Feature_Importance
Pclass              0.083386
Sex                 0.266363
Age                 0.266329
SibSp               0.047680
Parch               0.038059
Fare                0.266490
Embarked            0.031692


In [38]:
sp1 = 0
dp1 = 0
sp2 = 0
dp2 = 0
sp3 = 0
dp3 = 0
for passenger in dataset.index:
    if dataset.loc[passenger]['Survived'] == 1:
        if dataset.loc[passenger]['Pclass'] == 1:
            sp1 += 1
        elif dataset.loc[passenger]['Pclass'] == 2:
            sp2 += 1
        elif dataset.loc[passenger]['Pclass'] == 3:
            sp3 += 1
    elif dataset.loc[passenger]['Survived'] == 0:
        if dataset.loc[passenger]['Pclass'] == 1:
            dp1 += 1
        elif dataset.loc[passenger]['Pclass'] == 2:
            dp2 += 1
        elif dataset.loc[passenger]['Pclass'] == 3:
            dp3 += 1

print('Pclass1: ', sp1, ': ', dp1)
print('Pclass2: ', sp2, ': ', dp2)
print('Pclass3: ', sp3, ': ', dp3)

Pclass1:  136 :  80
Pclass2:  87 :  97
Pclass3:  119 :  372


In [19]:
csv1 = pd.read_csv('/home/user/Downloads/B1C2_node_estimation_pipeline_file_prepare_data_output_dataset.csv')
csv2 = pd.read_csv('/home/user/Downloads/B1R1_node_estimation_pipeline_file_prepare_data_output_dataset.csv')
csv3 = pd.read_csv('/home/user/Downloads/H2S1_node_estimation_pipeline_file_prepare_data_output_dataset.csv')

datasets = [csv1, csv2, csv3]

best_data = make_feature_selection(datasets, 'resected')
common = pd.concat([best_sample for best_sample in best_data], axis=0)
print(best_data[0].columns)

Index(['0.5-4Hz_cohy', '0.5-4Hz_pli2_unbiased', '4-7Hz_wpli', '7-14Hz_imcoh',
       '7-14Hz_ciplv', '14-30Hz_ciplv', '14-30Hz_wpli', '30-70Hz_ciplv',
       '30-70Hz_pli', 'pearson', 'envelope', 'resected'],
      dtype='object')


In [ ]:
subjects = pipeline(methods=['wpli', 'psd'],
                    freq_bands=[(0.5, 4), (4, 7), (7, 14), (14, 30), (30, 70)])


In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(a['B1C2']['X'], a['B1C2']['Y'])
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
#
# regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# regr.fit(x_train, y_train)
# y_pred = regr.predict(x_test)
#
# print(y_pred)
# print()
# print(y_test)

In [ ]:
# from nodestimation.mlearning.features import prepare_connectivity, prepare_data
# con = pkl.load(open(subjects[0].data['con'], 'rb'))

subjects_dir, subjects_ = find_subject_dir()
labels = mne.read_labels_from_annot('B1C2', parc='aparc', subjects_dir=subjects_dir)
label_names = [label.name for label in labels]
lh_labels = [name for name in label_names if name.endswith('lh')]
rh_labels = [name for name in label_names if name.endswith('rh')]

label_ypos_lh = list()

for name in lh_labels:
    idx = label_names.index(name)
    ypos = np.mean(labels[idx].pos[:, 1])
    label_ypos_lh.append(ypos)

try:
    idx = label_names.index('Brain-Stem')

except ValueError:
    pass

else:
    ypos = np.mean(labels[idx].pos[:, 1])
    lh_labels.append('Brain-Stem')
    label_ypos_lh.append(ypos)


lh_labels = [label for (yp, label) in sorted(zip(label_ypos_lh, lh_labels))]

rh_labels = [label[:-2] + 'rh' for label in lh_labels
             if label != 'Brain-Stem' and label[:-2] + 'rh' in rh_labels]


node_colors = [label.color for label in labels]

node_order = lh_labels[::-1] + rh_labels

node_angles = mne.viz.circular_layout(label_names, node_order, start_pos=90,
                              group_boundaries=[0, len(label_names) // 2])

In [ ]:
methods = [
    'coh',
    'imcoh',
    'plv',
    'ciplv',
    'ppc',
    'pli'
]

freq_bands = [
    '0.5-4Hz',
    '4-7Hz',
    '7-14Hz',
    '14-30Hz'
]

for method in methods:
    for freq_band in freq_bands:
        fig = plt.figure(num=None, figsize=(25, 25), facecolor='black')
        mne.viz.plot_connectivity_circle(con[freq_band][method]['con'][:, :, 0], label_names, n_lines=300,
                                         node_angles=node_angles, node_colors=node_colors,
                                         title='All-to-All Connectivity Epilepsy Condition ({} for {})'
                                         .format(method, freq_band), padding=8, fontsize_title=35, fontsize_colorbar=25,
                                         fontsize_names=20, fig=fig
                                         )

In [1]:
def return_none():
    return None

if return_none():
    print('None')
else:
    print('Else')



Else
